# Démonstration de Réplication d'Indice

Ce notebook démontre l'utilisation du framework de réplication d'indice pour comparer les méthodes physique et synthétique.

## Configuration de l'environnement

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Ajouter le dossier src au path pour importer les modules
sys.path.append('../src')

# Configurer les graphiques
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

# Créer les dossiers nécessaires s'ils n'existent pas déjà
os.makedirs('../logs', exist_ok=True)
os.makedirs('../data/raw', exist_ok=True)
os.makedirs('../data/processed', exist_ok=True)
os.makedirs('../results', exist_ok=True)

## 1. Téléchargement des données

Commençons par télécharger les données historiques pour un indice.

In [ ]:
from data.download_data import download_index_data, get_index_components, download_components_data, save_data

# Paramètres
index_name = 'CAC40'
index_ticker = '^FCHI'
start_date = '2020-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')

# Télécharger les données de l'indice
print(f"Téléchargement des données pour l'indice {index_name} ({index_ticker})")
index_data = download_index_data(index_ticker, start_date, end_date)

# Afficher les premières lignes
if index_data is not None:
    print(f"Données téléchargées avec succès: {index_data.shape[0]} jours")
    index_data.head()

In [ ]:
# Télécharger les composants de l'indice
components = get_index_components(index_name)
print(f"Composants de l'indice {index_name}: {len(components)} actions")
print(components[:10])  # Afficher les 10 premiers composants

In [ ]:
# Télécharger les données des composants (limiter à 5 composants pour la démo)
# En pratique, vous pourriez vouloir télécharger tous les composants
demo_components = components[:5]
components_data = download_components_data(demo_components, start_date, end_date)

# Afficher les premières lignes d'un composant
if components_data and demo_components[0] in components_data:
    print(f"Données téléchargées pour {len(components_data)} composants")
    components_data[demo_components[0]].head()

In [ ]:
# Sauvegarder les données
save_data(index_data, f"{index_name}_index", '../data/raw/indices')
save_data(components_data, f"{index_name}_components", '../data/raw/components')

print("Données sauvegardées avec succès.")

## 2. Traitement des données

Maintenant, traitons les données brutes pour préparer le backtest.

In [ ]:
from data.process_data import load_index_data, load_components_data, calculate_returns, create_price_matrix, create_return_matrix, process_weights_data, save_processed_data

# Charger les données brutes
raw_index_data = load_index_data(index_name, '../data/raw/indices')
raw_components_data = load_components_data(index_name, '../data/raw/components')

# Calculer les rendements de l'indice
if raw_index_data is not None:
    index_returns = calculate_returns(raw_index_data)
    print(f"Rendements calculés pour l'indice {index_name}")
    index_returns[['daily_return', 'cumulative_return']].head()

In [ ]:
# Créer la matrice de prix
if raw_components_data:
    price_matrix = create_price_matrix(raw_components_data)
    print(f"Matrice de prix créée: {price_matrix.shape}")
    price_matrix.head()

In [ ]:
# Créer la matrice de rendements
if raw_components_data:
    return_matrix = create_return_matrix(raw_components_data)
    print(f"Matrice de rendements créée: {return_matrix.shape}")
    return_matrix.head()

In [ ]:
# Créer les pondérations (égales pour la démo)
if raw_components_data:
    weights_df = process_weights_data(index_name, raw_components_data)
    print(f"Matrice de pondérations créée: {weights_df.shape}")
    weights_df.head()

In [ ]:
# Sauvegarder les données traitées
save_processed_data(index_returns, f"{index_name}_index_returns", '../data/processed/indices')
save_processed_data(price_matrix, f"{index_name}_price_matrix", '../data/processed/components')
save_processed_data(return_matrix, f"{index_name}_return_matrix", '../data/processed/components')
save_processed_data(weights_df, f"{index_name}_weights", '../data/processed/weights')

print("Données traitées sauvegardées avec succès.")

## 3. Réplication Physique

Testons d'abord la stratégie de réplication physique.

In [ ]:
from models.physical_replication import PhysicalReplication

# Paramètres
initial_capital = 1000000.0  # 1 million d'euros
management_fee = 0.0035      # 35 bps de frais de gestion
transaction_cost = 0.0020    # 20 bps de frais de transaction
rebalance_frequency = 'quarterly'

# Initialiser la réplication physique
physical = PhysicalReplication(
    index_name=index_name,
    start_date=start_date,
    end_date=end_date,
    data_dir='../data/processed',
    initial_capital=initial_capital,
    management_fee=management_fee,
    transaction_cost=transaction_cost,
    rebalance_frequency=rebalance_frequency
)

# Exécuter le backtest
physical_results = physical.run_backtest()

In [ ]:
# Afficher les métriques de performance
physical_metrics = physical.calculate_performance_metrics(physical_results)

print(f"Réplication Physique - Métriques de Performance")
print(f"Rendement Cumulé: {physical_metrics['portfolio']['cumulative_return']:.2%} (Ptf) vs {physical_metrics['index']['cumulative_return']:.2%} (Idx)")
print(f"Rendement Annualisé: {physical_metrics['portfolio']['annualized_return']:.2%} (Ptf) vs {physical_metrics['index']['annualized_return']:.2%} (Idx)")
print(f"Volatilité: {physical_metrics['portfolio']['volatility']:.2%} (Ptf) vs {physical_metrics['index']['volatility']:.2%} (Idx)")
print(f"Tracking Error: {physical_metrics['tracking_error']:.2%}")
print(f"Information Ratio: {physical_metrics['information_ratio']:.2f}")
print(f"Coûts de Transaction Totaux: {physical_metrics['total_transaction_costs']:.2f} €")
print(f"Nombre de Transactions: {physical_metrics['number_of_transactions']}")

In [ ]:
# Visualiser les résultats
physical.plot_results(physical_results)

## 4. Réplication Synthétique

Maintenant, testons la stratégie de réplication synthétique.

In [ ]:
from models.synthetic_replication import SyntheticReplication

# Paramètres
management_fee = 0.0025      # 25 bps de frais de gestion
swap_fee = 0.0015           # 15 bps de frais de swap
swap_reset_frequency = 'monthly'
risk_free_rate = 0.02       # 2% de taux sans risque

# Initialiser la réplication synthétique
synthetic = SyntheticReplication(
    index_name=index_name,
    start_date=start_date,
    end_date=end_date,
    data_dir='../data/processed',
    initial_capital=initial_capital,
    management_fee=management_fee,
    swap_fee=swap_fee,
    swap_reset_frequency=swap_reset_frequency,
    risk_free_rate=risk_free_rate
)

# Exécuter le backtest
synthetic_results = synthetic.run_backtest()

In [ ]:
# Afficher les métriques de performance
synthetic_metrics = synthetic.calculate_performance_metrics(synthetic_results)

print(f"Réplication Synthétique - Métriques de Performance")
print(f"Rendement Cumulé: {synthetic_metrics['portfolio']['cumulative_return']:.2%} (Ptf) vs {synthetic_metrics['index']['cumulative_return']:.2%} (Idx)")
print(f"Rendement Annualisé: {synthetic_metrics['portfolio']['annualized_return']:.2%} (Ptf) vs {synthetic_metrics['index']['annualized_return']:.2%} (Idx)")
print(f"Volatilité: {synthetic_metrics['portfolio']['volatility']:.2%} (Ptf) vs {synthetic_metrics['index']['volatility']:.2%} (Idx)")
print(f"Tracking Error: {synthetic_metrics['tracking_error']:.2%}")
print(f"Information Ratio: {synthetic_metrics['information_ratio']:.2f}")
print(f"Coûts de Swap Totaux: {synthetic_metrics['total_swap_costs']:.2f} €")
print(f"Nombre de Resets du Swap: {synthetic_metrics['number_of_swap_resets']}")

In [ ]:
# Visualiser les résultats
synthetic.plot_results(synthetic_results)

## 5. Comparaison des Stratégies

Enfin, comparons les deux stratégies côte à côte.

In [ ]:
from models.comparison import ReplicationComparison

# Paramètres pour la réplication physique
physical_params = {
    'management_fee': 0.0035,
    'transaction_cost': 0.0020,
    'rebalance_frequency': 'quarterly'
}

# Paramètres pour la réplication synthétique
synthetic_params = {
    'management_fee': 0.0025,
    'swap_fee': 0.0015,
    'swap_reset_frequency': 'monthly',
    'risk_free_rate': 0.02
}

# Initialiser la comparaison
comparison = ReplicationComparison(
    index_name=index_name,
    start_date=start_date,
    end_date=end_date,
    data_dir='../data/processed',
    initial_capital=initial_capital,
    physical_params=physical_params,
    synthetic_params=synthetic_params
)

# Exécuter la comparaison
comparison_results = comparison.run_comparison()

In [ ]:
# Visualiser la comparaison
comparison.plot_comparison(comparison_results)

In [ ]:
# Générer un rapport complet
report_dir = comparison.generate_report(comparison_results, '../results')
print(f"Rapport généré dans: {report_dir}")

## 6. Tests de Sensibilité

Enfin, effectuons quelques tests de sensibilité pour évaluer l'impact des paramètres clés.

In [ ]:
# Test de sensibilité sur les fréquences de rebalancement
rebalance_frequencies = ['monthly', 'quarterly', 'yearly']
physical_results_by_freq = {}
tracking_errors_by_freq = {}
transaction_costs_by_freq = {}

for freq in rebalance_frequencies:
    print(f"Test de la fréquence de rebalancement: {freq}")
    physical_test = PhysicalReplication(
        index_name=index_name,
        start_date=start_date,
        end_date=end_date,
        data_dir='../data/processed',
        initial_capital=initial_capital,
        management_fee=0.0035,
        transaction_cost=0.0020,
        rebalance_frequency=freq
    )
    
    results = physical_test.run_backtest()
    metrics = physical_test.calculate_performance_metrics(results)
    
    physical_results_by_freq[freq] = results
    tracking_errors_by_freq[freq] = metrics['tracking_error']
    transaction_costs_by_freq[freq] = metrics['total_transaction_costs']

In [ ]:
# Visualiser l'impact de la fréquence de rebalancement
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Tracking errors
tracking_errors = [tracking_errors_by_freq[freq] * 100 for freq in rebalance_frequencies]
ax1.bar(rebalance_frequencies, tracking_errors)
ax1.set_title('Tracking Error par Fréquence de Rebalancement')
ax1.set_xlabel('Fréquence de Rebalancement')
ax1.set_ylabel('Tracking Error (%)')
for i, value in enumerate(tracking_errors):
    ax1.text(i, value, f'{value:.2f}%', ha='center', va='bottom')

# Transaction costs
costs = [transaction_costs_by_freq[freq] for freq in rebalance_frequencies]
ax2.bar(rebalance_frequencies, costs)
ax2.set_title('Coûts de Transaction par Fréquence de Rebalancement')
ax2.set_xlabel('Fréquence de Rebalancement')
ax2.set_ylabel('Coûts Totaux (€)')
for i, value in enumerate(costs):
    ax2.text(i, value, f'{value:.2f} €', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Test de sensibilité sur les frais de gestion et de swap
fee_levels = [0.0010, 0.0025, 0.0050]  # 10, 25, 50 bps
synthetic_results_by_fee = {}
performance_by_fee = {}

for fee in fee_levels:
    print(f"Test avec frais de gestion et de swap: {fee:.2%}")
    synthetic_test = SyntheticReplication(
        index_name=index_name,
        start_date=start_date,
        end_date=end_date,
        data_dir='../data/processed',
        initial_capital=initial_capital,
        management_fee=fee,
        swap_fee=fee,
        swap_reset_frequency='monthly',
        risk_free_rate=0.02
    )
    
    results = synthetic_test.run_backtest()
    metrics = synthetic_test.calculate_performance_metrics(results)
    
    synthetic_results_by_fee[fee] = results
    performance_by_fee[fee] = metrics['portfolio']['cumulative_return']

In [ ]:
# Visualiser l'impact des frais sur la performance
fig, ax = plt.subplots(figsize=(10, 6))

fee_labels = [f'{fee:.2%}' for fee in fee_levels]
performance = [performance_by_fee[fee] * 100 for fee in fee_levels]
index_return = synthetic_metrics['index']['cumulative_return'] * 100

# Barres pour les différents niveaux de frais
ax.bar(fee_labels, performance)
ax.axhline(y=index_return, color='r', linestyle='--', label=f'Indice: {index_return:.2f}%')

ax.set_title('Performance Cumulée par Niveau de Frais (Réplication Synthétique)')
ax.set_xlabel('Niveau de Frais (Management + Swap)')
ax.set_ylabel('Rendement Cumulé (%)')
ax.legend()

for i, value in enumerate(performance):
    ax.text(i, value, f'{value:.2f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Conclusion

Ce notebook a démontré l'utilisation du framework de réplication d'indice pour comparer les méthodes physique et synthétique. Nous avons pu observer les différences de performance, de tracking error et de coûts entre les deux approches.

En résumé, nos résultats montrent que :

1. **Réplication Physique** :
   - Avantages : contrôle direct des actifs, transparence, perception des dividendes
   - Inconvénients : coûts de transaction plus élevés, complexité opérationnelle

2. **Réplication Synthétique** :
   - Avantages : coûts potentiellement plus bas, meilleure réplication (tracking error inférieur)
   - Inconvénients : risque de contrepartie, moins de transparence

Le choix entre ces deux méthodes dépend donc des objectifs spécifiques de l'investisseur, de son appétit pour le risque et des contraintes opérationnelles.